# Jsonifying Teamates csv's for Dashboard use

In [11]:
# Imports
import pandas as pd
from pathlib import Path
import json
import pprint as pp
import os
import us
import pymongo

 Sean's Gunsales data

In [12]:
# Store filepath in a variable
file = Path("../Sean/Resources/us_gun_sales_data_COMPLETE.csv")
gunsales_df = pd.read_csv(file)
gunsales_df

,Province_State,2023 Total Estimated Sales,Guns Per Capita
0,Alabama,487077,9.53
1,Alaska,80341,10.95
2,Arizona,407288,5.48
3,Arkansas,200279,6.53
4,California,1043421,2.68
5,Colorado,443800,7.55
6,Connecticut,120044,3.32
7,Delaware,47080,4.56
8,District of Columbia,4351,0.64
9,Florida,1316471,5.82


In [13]:
# Initialize an empty list to store dictionaries
gunsales_list = []

# Iterate over the DataFrame rows
for index, row in gunsales_df.iterrows():
    # Create a dictionary with the hospital name and other information
    gunsales_dict = {
        'State': row['Province_State'],
        'Gun Sales Info': row.drop(labels=['Province_State']).to_dict()
    }
    # Append the dictionary to the list
    gunsales_list.append(gunsales_dict)

# Convert the list of dictionaries to a JSON-formatted string
gunsales_json = json.dumps(gunsales_list, indent=4)

# Print the JSON-formatted string
print(gunsales_json)

[
    {
        "State": "Alabama",
        "Gun Sales Info": {
            "2023 Total Estimated Sales": 487077,
            "Guns Per Capita": 9.53
        }
    },
    {
        "State": "Alaska",
        "Gun Sales Info": {
            "2023 Total Estimated Sales": 80341,
            "Guns Per Capita": 10.95
        }
    },
    {
        "State": "Arizona",
        "Gun Sales Info": {
            "2023 Total Estimated Sales": 407288,
            "Guns Per Capita": 5.48
        }
    },
    {
        "State": "Arkansas",
        "Gun Sales Info": {
            "2023 Total Estimated Sales": 200279,
            "Guns Per Capita": 6.53
        }
    },
    {
        "State": "California",
        "Gun Sales Info": {
            "2023 Total Estimated Sales": 1043421,
            "Guns Per Capita": 2.68
        }
    },
    {
        "State": "Colorado",
        "Gun Sales Info": {
            "2023 Total Estimated Sales": 443800,
            "Guns Per Capita": 7.55
        }
    },
   

In [14]:
# Path to the resources folder
file_path = os.path.join('Resources', 'cleaned_gunsales.json')

# Write the list of dictionaries to a JSON file in the resources folder
with open(file_path, 'w') as json_file:
    json.dump(gunsales_list, json_file, indent=4)

print(f"JSON data has been written to {file_path}")

JSON data has been written to Resources/cleaned_gunsales.json
